# ***Proyecto EDA: Airbnb*** 
## ***Limpieza y Transformación***

**Autor:** Óscar Fernández-Chinchilla López

![Logo Airbnb](../images/airbnb-logo.png)

# 📋 Índice

1. [Introducción y objetivos de la limpieza](#introduccion)
2. [Carga del dataset original e importación de las librerias](#carga)
3. [Limpieza de columnas](#limpieza_col)
3. [Tratamiento de valores nulos](#nulos)
   - [Visualización de nulos](#vis-nulos)
   - [Imputación / Eliminación](#imputacion)
4. [Corrección de tipos de datos](#tipos)
5. [Estandarización de variables categóricas](#categoricas)
6. [Corrección de incoherencias en los datos](#incoherencias)
7. [Detección y tratamiento de outliers](#outliers)
8. [Comprobación final del dataset](#comprobacion)
9. [Guardado del dataset limpio](#guardado)

<a id="introduccion"></a>
## 1️⃣ Introducción y objetivos de la limpieza <a id="introduccion"></a>


Para poder realizar un análisis fiable del dataset y extraer conclusiones realmente
representativas, es necesario corregir los problemas identificados durante la fase de
exploración. Por ello, en este notebook vamos a llevar a cabo un proceso de limpieza y
transformación del dataset.

## 🎯 Objetivos de la limpieza:

- **Renombrar y limpiar** los nombres de las columnas para facilitar su manejo.
- **Eliminar** duplicados que puedan generar ruido o sesgos en el análisis.
- **Tratar** los valores nulos, tomando decisiones justificadas en cada caso:
  - **Eliminar** columnas o filas con demasiados nulos o información irrelevante
  - **Imputar** valores cuando la información sea importante mantenerla
- **Corregir** los tipos de datos que no sean coherentes (ej.: `price` como `object` → `float`)
- **Estandarizar** categorías con errores ortográficos o incoherencias (ej.: boroughs mal escritos)
- **Detectar y ajustar outliers** para evitar que afecten de forma negativa a análisis posteriores
- **Formatear** las fechas correctamente para futuros cálculos temporales

---

Una vez completado este proceso, obtendremos un dataset **limpio y preparado** para continuar con
el análisis y la visualización avanzada.



# 2️. Carga del dataset original e importación de librerías <a id="carga"></a>

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Cargamos el dataset original
airbnb_df = pd.read_csv("../data/Airbnb_Open_Data.csv", low_memory=False)

# Creamos una copia para trabajar con ella y mantener el original intacto en caso de error
airbnb_clean = airbnb_df.copy()

<a id="normalización"></a>
# 3️⃣ Normalización de las columnas

### Para comenzar con el proceso de limpieza, lo primero que haré será realizar la normalización de las columnas del dataset
Esto se debe a que para poder tratar de mejor manera los datos y poder acceder a las columnas, es mejor normalizarlas al inicio por si contienen espacios, simbolos, mayusculas o minusculas incoherentes entre otras cosas.

In [ ]:
# Aqui definimos una función destinada a normalizar los nombres de las columnas para que sean más legibles.
# En estas funciones, siempre trabajaremos sobre una copia del df pasado por parámetro
def normalize_column_names(df):
    df_cleaned = df.copy()
    df_cleaned.columns = (
        df_cleaned.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace('[^0-9a-zA-Z_]', '', regex=True)
    )
    return df_cleaned

In [38]:
# Una vez tenemos la función creada, la utilizamos con la copia del dataset original para poder empezar a trabajar
airbnb_clean = normalize_column_names(airbnb_clean)
airbnb_clean.columns

Index(['id', 'name', 'host_id', 'host_identity_verified', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'lat', 'long', 'country',
       'country_code', 'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'price', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'review_rate_number', 'calculated_host_listings_count',
       'availability_365', 'house_rules', 'license'],
      dtype='object')

#### Como se puede observar en el anterior output, todos los nombres de las columnas del dataset aparecen en minúscula, sin espacios en los extremos y con separadores de "_" entre palabras.

# 4. Proceso de limpieza

<a id="normalización"></a>
## 4.1 Limpieza de columnas monetarias

#### En este apartado procederé con la limpieza de las columnas monetarias que tiene el dataset. En este caso hacen referencia solo a las columnas de Precio y Tarifa de servicio

In [39]:

#Aqui definiremos una función para limpiar las columnas monetarias del dataset, las cuales son pasadas por parámetro junto al dataframe
def clean_monetary_columns(df,monetary_columns):
    df_cleaned = df.copy()
    
    # Bucle que itera en la lista pasada por parametro
    for col in monetary_columns:
        # Comprobamos que la columna existe en el dataframe, y en el caso de que exista realizamos las siguientes operaciones
        if col in df_cleaned.columns:
            df_cleaned[col] = (
                df_cleaned[col].astype(str)       # Convierto el objeto a String para poder operar
                .str.replace(r'[^0-9\.\-]', '', regex=True)         # Elimino los simbolos cambiandolos por un espacio en blanco
                .str.strip()        # Los elimino de los laterales
            )
            # Convertimos las columnas a tipo numérico
            df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors="coerce")
    return df_cleaned

In [40]:
# Creo la lista con los nombres de las columnas a limpiar
monetary_columns = ["price", "service_fee"]
airbnb_clean = clean_monetary_columns(airbnb_clean,monetary_columns)

#### Con las siguientes celdas comprobamos si la limpieza y la transformación de las columnas se ha realizado correctamente

In [ ]:
airbnb_clean.price.info

<bound method Series.info of 0          966.0
1          142.0
2          620.0
3          368.0
4          204.0
           ...  
102594     844.0
102595     837.0
102596     988.0
102597     546.0
102598    1032.0
Name: price, Length: 102599, dtype: float64>

In [7]:
airbnb_clean.service_fee.info

<bound method Series.info of 0         193.0
1          28.0
2         124.0
3          74.0
4          41.0
          ...  
102594    169.0
102595    167.0
102596    198.0
102597    109.0
102598    206.0
Name: service_fee, Length: 102599, dtype: float64>

<a id="categorian"></a>
## 4.2 Limpieza de columnas categóricas

#### En el siguiente apartado definimos una función para limpiar y transformar las columnas categoricas seleccionadas y observadas previamente en el Notebook de exploración.
#### En cuanto a la columna de los barrios pequeños del dataset, decido limpiarlos porque algunos valores contienen más de 1 palabra separadas por comas, con valores erroneos de las variables. Ejemploç: "Chealse, Staten Island". Chealse no forma parte del Borough de Staten Island, de misma manera que este patrón se repite, por lo que decido quedarme con la parte de la izquierda de la coma y descartar la parte posterior.

In [45]:
def clean_categorical_values(df):
    df_cleaned = df.copy()
    # Ahora limpiaremos y normalizaremos los valores categóricos de la columna neighbourhood_group
    if "neighbourhood_group" in df_cleaned.columns:
        df_cleaned["neighbourhood_group"] =  df_cleaned["neighbourhood_group"].replace(
            {
                "brookln" : "Brooklyn",
                "manhatan": "Manhattan"
            }
        )
        # Reemplazar valores nulos por "Unknown"
        df_cleaned["neighbourhood_group"] = df_cleaned["neighbourhood_group"].fillna("Unknown")
    
    # Inputamos los nulos de la columna "host_identity_verified"
    if "host_identity_verified" in df_cleaned.columns:
        # Aqui decido remplazar el valor de verified por confirmed para la comparativa en el análisis avanzado
        df_cleaned["host_identity_verified"] = df_cleaned["host_identity_verified"].replace({
        "verified": "confirmed"
        })
        df_cleaned["host_identity_verified"] = df_cleaned["host_identity_verified"].fillna("Unknown")
    
    # Inputamos también los nombres de la columna "neighbourhood"
    df_cleaned["neighbourhood"] = (
        df_cleaned["neighbourhood"]
        .astype(str)
        .str.split(",", n=1).str[0]
        .str.strip()
    )
    
    # Convertimos la columna "instant_bookable" a Booleano 
    if "instant_bookable" in df_cleaned.columns:
            df_cleaned["instant_bookable"] =  df_cleaned["instant_bookable"].replace(
            {
                "False" : False,
                "True": True
            }
            )
            
    df_cleaned["instant_bookable"] = df_cleaned["instant_bookable"].astype("boolean")
    
    # Convertimos columnas a category
    categorical_cols = [
        "host_identity_verified",
        "neighbourhood_group",
        "neighbourhood",
        "country",
        "country_code",
        "cancellation_policy",
        "room_type"
    ]

    for col in categorical_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = df_cleaned[col].astype("category")
    return df_cleaned
    

In [46]:
airbnb_clean = clean_categorical_values(airbnb_clean)

In [47]:
airbnb_clean.dtypes

id                                   int64
name                                object
host_id                              int64
host_identity_verified            category
host_name                           object
neighbourhood_group               category
neighbourhood                     category
lat                                float64
long                               float64
country                           category
country_code                      category
instant_bookable                   boolean
cancellation_policy               category
room_type                         category
construction_year                  float64
price                              float64
service_fee                        float64
minimum_nights                     float64
number_of_reviews                  float64
last_review                         object
reviews_per_month                  float64
review_rate_number                 float64
calculated_host_listings_count     float64
availabilit

rangos (extremos) y duplicados

<a id="fechas"></a>
## 4.3 Formateo de las fechas de ultima review (por anuncio)

#### En el siguiente apartado se formatearán las fechas de la columna "last_review" para que obtenga un formato datetime y asi porder acceder a valores como el año, mes y dia en el análisis

In [53]:
def format_dates(df):
    df_cleaned = df.copy()
    # Lo siguiente que realizaremos será formatear las fechas de la columna "last review"
    if "last_review" in df_cleaned.columns:
        df_cleaned['last_review'] = pd.to_datetime(df_cleaned['last_review'], errors='coerce')
    return df_cleaned

In [54]:
airbnb_clean = format_dates(airbnb_clean)
df = airbnb_clean["last_review"]
df

0        2021-10-19
1        2022-05-21
2               NaT
3        2019-07-05
4        2018-11-19
            ...    
102594          NaT
102595   2015-07-06
102596          NaT
102597   2015-10-11
102598          NaT
Name: last_review, Length: 102599, dtype: datetime64[ns]

#### Podemos observar que algunas filas contienen valores como Nat, las cuales he decidido no limpiar ya que nos puede dar información valiosa. Si un anuncio no tiene una fecha de ultima reseña, puede ser porque sea un anuncio nuevo.

<a id="numericas"></a>
## 4.4 Transformación de columnas numéricas a Float o Integer

#### En este apartado definimos una función que recogerá por parametro el dataframe, y las columnas a convertir Integer o Float según la intención que tengamos para analizarlas.

In [55]:
def convert_numeric_types(df, int_cols=None, float_cols=None):
    df_cleaned = df.copy()

    # Columnas que deben ser enteros
    if int_cols:
        for col in int_cols:
            if col in df_cleaned.columns:
                df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors="coerce").astype("Int64")

    # Columnas que deben ser float
    if float_cols:
        for col in float_cols:
            if col in df_cleaned.columns:
                df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors="coerce")

    return df_cleaned

In [57]:
airbnb_clean = convert_numeric_types(
    airbnb_clean,
    int_cols=["number_of_reviews", "minimum_nights", "calculated_host_listings_count", "availability_365","construction_year"],
    float_cols=["reviews_per_month"]
)
airbnb_clean.dtypes

id                                         int64
name                                      object
host_id                                    int64
host_identity_verified                  category
host_name                                 object
neighbourhood_group                     category
neighbourhood                           category
lat                                      float64
long                                     float64
country                                 category
country_code                            category
instant_bookable                         boolean
cancellation_policy                     category
room_type                               category
construction_year                          Int64
price                                    float64
service_fee                              float64
minimum_nights                             Int64
number_of_reviews                          Int64
last_review                       datetime64[ns]
reviews_per_month   

# 5 . Eliminación de duplicados

In [61]:
airbnb_clean.duplicated().sum()

np.int64(541)

#### Existen 541 filas duplicadas en el dataset, un porcentaje bastante bajo

In [62]:
airbnb_clean.drop_duplicates(inplace=True)

#### Con la celda anterior eliminamos los duplicados dejando solo la primera instancia que aparezca de cada fila duplicada

In [63]:
airbnb_clean.duplicated().sum()

np.int64(0)

### Una vez eliminados, comprobamos que ya no haya más duplicados, dando el resultado 0

#### Se detectaron 541 filas duplicadas, las cuales fueron eliminadas para evitar distorsiones estadísticas en los análisis posteriores. Tras la eliminación, no existen registros duplicados en el dataset
#### Se eliminaron los duplicados manteniendo la primera aparición de cada fila, ya que estas correspondían a registros idénticos y no aportaban información adicional.

# 6. Tratamiento de los rangos observados en la exploración

In [68]:
# availability_365 no puede ser > 365
airbnb_clean.loc[airbnb_clean["availability_365"] > 365, "availability_365"] = pd.NA
airbnb_clean.loc[airbnb_clean["availability_365"] < 0, "availability_365"] = pd.NA

# minimum_nights no puede ser <= 0
airbnb_clean.loc[airbnb_clean["minimum_nights"] <= 0, "minimum_nights"] = pd.NA

# construction_year imposible (antes de 1800 o > año actual)
current_year = pd.Timestamp.now().year
airbnb_clean.loc[airbnb_clean["construction_year"] < 1800, "construction_year"] = pd.NA
airbnb_clean.loc[airbnb_clean["construction_year"] > current_year, "construction_year"] = pd.NA

In [69]:
print("availability_365:", airbnb_clean["availability_365"].min(), "-", airbnb_clean["availability_365"].max())
print("minimum_nights:", airbnb_clean["minimum_nights"].min(), "-", airbnb_clean["minimum_nights"].max())
print("construction_year:", airbnb_clean["construction_year"].min(), "-", airbnb_clean["construction_year"].max())

availability_365: 0 - 365
minimum_nights: 1 - 5645
construction_year: 2003 - 2022
